In [1]:
import pandas as pd
import numpy as np
import torch

In [9]:
# Pseudo algorithm
# for score 5.5:
#   - for P1 th-1 b/w 0.5, 0.1, 0.05:
#     - for P3 th-3 b/w 0.5, 0.1, 0.05:
#       - for P4 th-4 b/w 0.5, 0.1, 0.05:
#         - for P5 th-5 b/w 0.5, 0.1, 0.05:
#           - calculate rmse
#           - pick the lowest one
# 
# At the end we have the threshold, for all parts for score 5.5
# Then do the same thing for score 5, by fixing threshold for 5.5 - found above
# ...continue

In [2]:
def corn_forward_logits(raw_logits: torch.Tensor):
    B, K1 = raw_logits.shape
    shifted_logits = torch.zeros_like(raw_logits)
    shifted_logits[:, 0] = raw_logits[:, 0]
    for k in range(1, K1):
        log_odds_prev = shifted_logits[:, k-1]
        shifted_logits[:, k] = raw_logits[:, k] + log_odds_prev
    return shifted_logits

In [3]:
def corn_inference(raw_logits: torch.Tensor, score2optimize, trial_threshold, dict_fixedThresholds):
    """
    raw_logits: shape (B, K-1)
    Return integer predictions in [0..K].
    We'll do the same shifting, then threshold each probability at 0.5.
    """
    z = corn_forward_logits(raw_logits)  # shift
    p = torch.sigmoid(z)
    
    threshold_list = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
    for key in dict_fixedThresholds.keys():
        threshold_list[key] = dict_fixedThresholds[key]
    
    idx_optimize = int(score2optimize*2 - 4 - 1)
    threshold_list[idx_optimize] = trial_threshold
    
    threshold_tensor = torch.tensor(threshold_list)
    
    pass_mask = raw_logits > threshold_tensor
                
    preds = pass_mask.sum(dim=1)
    return preds

In [4]:
from tqdm.auto import tqdm

dict_Thresholds_p1 = {}
dict_Thresholds_p3 = {}
dict_Thresholds_p4 = {}
dict_Thresholds_p5 = {}

# Example: thresholds from 0.0 to 1.0 in steps of 0.05
th_values = [round(x, 2) for x in np.arange(0.0, 1.001, 0.1)]
    
min_rmse = 0.8
good_thresholds = []

path = "/m/triton/work/porwala1/slate_models/Baseline_CornLoss_TrainModel_preds_withProbs"

P1_csv = pd.read_csv(path + "/dev-preds-P1-mid.csv") #Get from CSV
pred_logits_1_df = P1_csv.drop(columns=['score', 'actualScore', 'submissionID'])
pred_logits_1 = pred_logits_1_df.to_numpy()
logits_t_1 = torch.from_numpy(pred_logits_1)

P3_csv = pd.read_csv(path + "/dev-preds-P3-mid.csv") #Get from CSV
pred_logits_3_df = P3_csv.drop(columns=['score', 'actualScore', 'submissionID'])
pred_logits_3 = pred_logits_3_df.to_numpy()
logits_t_3 = torch.from_numpy(pred_logits_3)

P4_csv = pd.read_csv(path + "/dev-preds-P4-mid.csv") #Get from CSV
pred_logits_4_df = P4_csv.drop(columns=['score', 'actualScore', 'submissionID'])
pred_logits_4 = pred_logits_4_df.to_numpy()
logits_t_4 = torch.from_numpy(pred_logits_4)

P5_csv = pd.read_csv(path + "/dev-preds-P5-mid.csv") #Get from CSV
pred_logits_5_df = P5_csv.drop(columns=['score', 'actualScore', 'submissionID'])
pred_logits_5 = pred_logits_5_df.to_numpy()
logits_t_5 = torch.from_numpy(pred_logits_5)

actualScorePath = f"/m/triton/scratch/elec/t405-puhe/c/sandi2025/reference-materials/sla-marks/dev-sla-overall.tsv"
overallActualScore = pd.read_csv(actualScorePath, names=["speakerID", "score"], sep="\t")

scores_to_optimize = [5.5, 5.0, 4.5, 4.0, 3.5, 3.0, 2.5]

all_good_thresholds = []  # Keep track of best combos found

# Outer loop over the 7 target scores
for score in tqdm(scores_to_optimize, desc="Optimizing each target score"):
    
    # Re-initialize for this score
    min_rmse = float("inf")
    best_threshold = [0.5, 0.5, 0.5, 0.5]
    
    # Loop over all 4 threshold combinations for P1, P3, P4, P5
    for th_1 in tqdm(th_values, leave=False, desc=f"Trying thresholds P1 for score={score}"):
        
        # Part 1 inference
        preds_t_1 = corn_inference(logits_t_1, score, th_1, dict_Thresholds_p1)
        final_preds_1 = preds_t_1.cpu().numpy()
        final_preds_rescaled_1 = final_preds_1 / 2 + 2
        
        for th_3 in tqdm(th_values, leave=False, desc=f"P3 for score={score}"):
            
            preds_t_3 = corn_inference(logits_t_3, score, th_3, dict_Thresholds_p3)
            final_preds_3 = preds_t_3.cpu().numpy()
            final_preds_rescaled_3 = final_preds_3 / 2 + 2
            
            for th_4 in tqdm(th_values, leave=False, desc=f"P4 for score={score}"):
                
                preds_t_4 = corn_inference(logits_t_4, score, th_4, dict_Thresholds_p4)
                final_preds_4 = preds_t_4.cpu().numpy()
                final_preds_rescaled_4 = final_preds_4 / 2 + 2
                
                for th_5 in tqdm(th_values, leave=False, desc=f"P5 for score={score}"):
                    
                    preds_t_5 = corn_inference(logits_t_5, score, th_5, dict_Thresholds_p5)
                    final_preds_5 = preds_t_5.cpu().numpy()
                    final_preds_rescaled_5 = final_preds_5 / 2 + 2
                    
                    # Calculate overall predicted score = mean of the 4 part-scores
                    predOverallScore = np.mean([
                        final_preds_rescaled_1,
                        final_preds_rescaled_3,
                        final_preds_rescaled_4,
                        final_preds_rescaled_5
                    ], axis=0)
                    
                    # Calculate RMSE with actual overall scores
                    rmse = np.sqrt(
                        np.mean((overallActualScore["score"].to_numpy() - predOverallScore) ** 2)
                    )
                    
                    # Update best threshold if this is better
                    if rmse < min_rmse:
                        min_rmse = rmse
                        best_threshold = [th_1, th_3, th_4, th_5]
                        all_good_thresholds.append([score, th_1, th_3, th_4, th_5, rmse])
    
    # After testing all threshold combos for the current score,
    # store the best threshold into the dictionaries, so we "lock" them in:
    idx_score = int(score * 2 - 4 - 1)
    
    dict_Thresholds_p1[idx_score] = best_threshold[0]
    dict_Thresholds_p3[idx_score] = best_threshold[1]
    dict_Thresholds_p4[idx_score] = best_threshold[2]
    dict_Thresholds_p5[idx_score] = best_threshold[3]
    
    print(f"Best thresholds for score={score}: {best_threshold} with RMSE={min_rmse:.4f}")

# Done!
print("All done. Final locked thresholds for each score index:")
print("P1:", dict_Thresholds_p1)
print("P3:", dict_Thresholds_p3)
print("P4:", dict_Thresholds_p4)
print("P5:", dict_Thresholds_p5)

Optimizing each target score:   0%|          | 0/7 [00:00<?, ?it/s]

Trying thresholds P1 for score=5.5:   0%|          | 0/11 [00:00<?, ?it/s]

P3 for score=5.5:   0%|          | 0/11 [00:00<?, ?it/s]

P4 for score=5.5:   0%|          | 0/11 [00:00<?, ?it/s]

P5 for score=5.5:   0%|          | 0/11 [00:00<?, ?it/s]

...

P5 for score=2.5:   0%|          | 0/11 [00:00<?, ?it/s]

P5 for score=2.5:   0%|          | 0/11 [00:00<?, ?it/s]

P5 for score=2.5:   0%|          | 0/11 [00:00<?, ?it/s]

P5 for score=2.5:   0%|          | 0/11 [00:00<?, ?it/s]

Best thresholds for score=2.5: [0.7, 0.8, 0.8, 0.9] with RMSE=0.3726
All done. Final locked thresholds for each score index:
P1: {6: 0.1, 5: 0.2, 4: 0.5, 3: 0.7, 2: 0.9, 1: 0.6, 0: 0.7}
P3: {6: 0.1, 5: 0.5, 4: 0.6, 3: 0.8, 2: 0.7, 1: 0.6, 0: 0.8}
P4: {6: 0.4, 5: 0.2, 4: 0.4, 3: 0.6, 2: 0.4, 1: 0.8, 0: 0.8}
P5: {6: 0.5, 5: 0.1, 4: 0.3, 3: 0.7, 2: 0.6, 1: 0.9, 0: 0.9}


In [5]:
dict_Thresholds_p1, dict_Thresholds_p3, dict_Thresholds_p4, dict_Thresholds_p5

({6: 0.1, 5: 0.2, 4: 0.5, 3: 0.7, 2: 0.9, 1: 0.6, 0: 0.7},
 {6: 0.1, 5: 0.5, 4: 0.6, 3: 0.8, 2: 0.7, 1: 0.6, 0: 0.8},
 {6: 0.4, 5: 0.2, 4: 0.4, 3: 0.6, 2: 0.4, 1: 0.8, 0: 0.8},
 {6: 0.5, 5: 0.1, 4: 0.3, 3: 0.7, 2: 0.6, 1: 0.9, 0: 0.9})

In [7]:
min_rmse

0.37256630374328203